In [44]:
import nltk
from nltk.corpus import words, brown
nltk.download('words')
word_dictionary = list(set(words.words()))

from collections import Counter

[nltk_data] Downloading package words to /Users/apple/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [47]:
from nltk import bigrams 
from nltk.stem import  WordNetLemmatizer 
from collections import Counter
from nltk.corpus import stopwords
import string
import re
import json
 
lemmatiser = WordNetLemmatizer()

#abbrevs={"won't":"will not","i'm":"i am","don't":"do not","doesn't":"does not","isn't":"is not","i'll":"i will"}

punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['RT','rt','VIA', 'via']

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
def normalization(s,arra):
    tokens = preprocess(s)
    count = 0
    count2 =0
    str = []
    for token in tokens:
        #canonicalization of text
        value =lemmatiser.lemmatize(token, pos="v")
        value1 =lemmatiser.lemmatize(value)
        value2 = value1.encode('utf8') 
        #converting the entire text into lowercase
        value2 = value2.lower() 
        #expanding abbreviations
        """
        for abbrev in abbrevs:
            value2= value2.replace(abbrev,abbrevs[abbrev])
        """
        pattern = re.compile('http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+' )
        if not pattern.match(value2):
            if value2 not in arra: 
                str.append(value2)
        
    
    return str
def tokenize(s):
    return tokens_re.findall(s)

    return s
def preprocess(s, lowercase=False): 
    tokens = tokenize(s) 
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens 

In [3]:
for alphabet in "bcdefghjklmnopqrstuvwxyz":
    word_dictionary.remove(alphabet)



In [4]:
def split_hashtag_to_words_all_possibilities(hashtag):
    all_possibilities = []
        
    split_posibility = [hashtag[:i] in word_dictionary for i in reversed(range(len(hashtag)+1))]
    possible_split_positions = [i for i, x in enumerate(split_posibility) if x == True]

    for split_pos in possible_split_positions:
        split_words = []
        word_1, word_2 = hashtag[:len(hashtag)-split_pos], hashtag[len(hashtag)-split_pos:]

        if word_2 in word_dictionary:
            split_words.append(word_1)
            split_words.append(word_2)
            all_possibilities.append(split_words)

            another_round = split_hashtag_to_words_all_possibilities(word_2)
 
            if len(another_round) > 0:
                 all_possibilities = all_possibilities + [[a1] + a2 for a1, a2, in zip([word_1]*len(another_round), another_round)]
        else:
            another_round = split_hashtag_to_words_all_possibilities(word_2)
 
            if len(another_round) > 0:
                all_possibilities = all_possibilities + [[a1] + a2 for a1, a2, in zip([word_1]*len(another_round), another_round)]

    return all_possibilities

In [ ]:
split_tag=[]
with open('property-view-perth.json' ,'r') as nf:
    for line in nf:
        split = split_hashtag_to_words_all_possibilities(ins)
        print(split[0])
        
        

In [51]:
value = split_hashtag_to_words_all_possibilities("house")

In [52]:
print(value)

[['ho', 'use'], ['ho', 'us', 'e'], ['ho', 'u', 'se'], ['ho', 'u', 's', 'e'], ['h', 'o', 'use'], ['h', 'o', 'us', 'e'], ['h', 'o', 'u', 'se'], ['h', 'o', 'u', 's', 'e']]


In [53]:
print(value[0])

['ho', 'use']


In [49]:
count_all = Counter()
count_bigram = Counter()
f = open('property-view-perth.json', 'r')
lines = f.read()

In [55]:
splittoken=[]
token = normalization(lines.decode('unicode_escape').encode('ascii','ignore'),stop)
for tag in token:
    value = split_hashtag_to_words_all_possibilities(tag)
    print value
    firsttoken = value[0]
    splittoken.append(firsttoken)
    print(value[0])
count_all.update(splittoken)

[]


IndexError: list index out of range

In [ ]:
print(count_all.most_common(50))